In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
from sklearn import metrics

import matplotlib.pyplot as plt
import random as random
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import zero_one_loss
import sklearn.linear_model
from sklearn import svm
from sklearn.metrics import roc_curve, auc, precision_recall_curve,average_precision_score

from sklearn import linear_model #TODO: more models
from sklearn import tree
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import naive_bayes
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from scipy import sparse

from sklearn.datasets import make_classification
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.metrics import roc_curve
from sklearn.preprocessing import scale

import statsmodels.api as sm

from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

//anaconda/envs/py27/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Reading the input dataset

In [2]:
curr_dir = !pwd
input_path = curr_dir[0]+"/../9.Features_exploration/binding_df/10/"
#filename = "positions_features_08.08.17.csv" # Old version
filename = "positions_features_10.31.17.csv"
bind_scores_num = 10

features_all = pd.read_csv(input_path+filename, sep='\t', index_col=0)
features_cols = features_all.columns[1:-bind_scores_num] #removing binding scores and domain name
#ligands = ["dna", "rna", "peptide", "ion", "metabolite"]
ligands = ["dna", "dnabase", "dnabackbone", "rna", "rnabase", "rnabackbone", "peptide", "ion", "metabolite"]

### Checking input features doesn't have NaNs

In [3]:
for col in features_cols:
    nan_idx = np.where( np.isnan(features_all[col].tolist()) == True)[0]
    if (len(nan_idx) > 0):
        print col+" has NaNs"

### Checking input features doesn't have Infs

In [4]:
for col in features_cols:
    inf_idx = np.where(np.isinf(features_all[col].tolist()) == True)[0]
    if (len(inf_idx) > 0):
        print col+" has Inf"

### Dataset of negative examples

In [5]:
non_binding_positions = features_all[features_all["max_binding_score"] == 0]
non_binding_positions = non_binding_positions.loc[:,features_cols]

### Datasets of positive examples by ligand

In [7]:
bind_th = 0.1
ligands_features_df = {}

for ligand in ligands:
    score_col_str = ligand+"_binding_score"
    ligand_binding_df = features_all[features_all[score_col_str] >= bind_th]
    ligands_features_df[ligand] = ligand_binding_df.loc[:,features_cols]

### Functions to run models

In [8]:
def log_predict_threshold(probs, predict_th, class_idx):
    
    predicted = []
    for i in range(len(probs)):
        if (probs[i][class_idx] >= predict_th):
            predicted.append(1)
        else:
            predicted.append(0)
        
    return np.array(predicted)

In [9]:
def predict_threshold(probs, predict_th):
    
    predicted = []
    for i in range(len(probs)):
        if (probs[i] >= predict_th):
            predicted.append(1)
        else:
            predicted.append(0)
        
    return np.array(predicted)

In [10]:
def sample(iterator, k):
    """
    Samples k elements from an iterable object.

    :param iterator: an object that is iterable
    :param k: the number of items to sample
    """
    # fill the reservoir to start
    result = [next(iterator) for _ in range(k)]

    n = k - 1
    for item in iterator:
        n += 1
        s = random.randint(0, n)
        if s < k:
            result[s] = item

    return result

In [11]:
# Down-sample a balanced set with largest possible number of positives
def get_sample(ligand_bind_features,non_binding_positions):
    negative_idx = range(0,len(non_binding_positions))
    sampled_negative_idx = sample(iter(negative_idx), len(ligand_bind_features))
    X = pd.concat([ligand_bind_features, non_binding_positions.iloc[sampled_negative_idx]])

    y = [1] * ligand_bind_features.shape[0]
    y.extend([0] * ligand_bind_features.shape[0])
    y = np.array(y)
    return [X,y]

### Test features and create ranking

In [12]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import ExtraTreesClassifier


# List of features
features_list = np.array([c.encode('ascii','ignore') for c in ligands_features_df['dna'].columns])
# Rankings by ligand/test
ranking = defaultdict(list)
# Dataframe to hold feature selection results - 1 indicates retention at the threshold
features_df = pd.DataFrame(columns=features_list)

for ligand in ligands:
    # Down-sample
    X,y = get_sample(ligands_features_df[ligand],non_binding_positions)
    
    # Feature selection models
    var_model = VarianceThreshold(threshold=(.3)).fit(X)
    kbest_model = SelectKBest(k=200).fit(X,y)
    trees_model = ExtraTreesClassifier().fit(X,y)
    
    # Store results
    #features_df.loc["Vartest_0.1_"+ligand] = np.asarray(var_model.get_support()).astype(int)
    #features_df.loc["kbest_200_"+ligand] = np.asarray(kbest_model.get_support()).astype(int)
    #features_df.loc["Trees_"+ligand,:] = np.asarray(trees_model.feature_importances_ > 1.0/len(features_list)).astype(int)
    
    # Extract feature scores and remove NaNs
    # Variances for var; ANOVA f-value for kbest; Relative importances for trees
    var = np.asarray(var_model.variances_)
    var[np.isnan(var)] = -1
    kbest = np.asarray(kbest_model.scores_)
    kbest[np.isnan(kbest)] = -1
    trees = np.asarray(trees_model.feature_importances_)
    trees[np.isnan(trees)] = -1

    # Create ranking
    ranking[ligand+"_var"] = features_list[np.argsort(var)][::-1]
    ranking[ligand+"_kbest"] = features_list[np.argsort(kbest)][::-1]
    ranking[ligand+"_trees"] = features_list[np.argsort(trees)][::-1]

    # Store all scores\n",
    features_df.loc["var_"+ligand,:] = var
    features_df.loc["kbest_"+ligand,:] = kbest
    features_df.loc["trees_"+ligand,:] = trees

//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [304 305 306 307 309 310 311 312 314 315 316 317 319 320 321 322 324 325
 326 327 329 330 331 332 333 334 335 336 337 338] are constant.
  UserWarning)
//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [176 197 304 305 306 307 309 310 311 312 314 315 316 317 319 320 321 322
 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338] are constant.
  UserWarning)


In [13]:
features_df

,avg_maf_all,avg_maf_altered,maf_hist_0-0.001,maf_hist_0.001-0.005,maf_hist_0.005-0.01,maf_hist_0.01-0.02,maf_hist_0.02-0.04,maf_hist_0.04-0.06,maf_hist_0.06-0.08,maf_hist_0.08-0.1,...,sub_func_group_trans_4-2,sub_func_group_trans_4-3,sub_func_group_trans_4-4,sub_func_group_trans_4-5,sub_func_group_trans_5-0,sub_func_group_trans_5-1,sub_func_group_trans_5-2,sub_func_group_trans_5-3,sub_func_group_trans_5-4,sub_func_group_trans_5-5
var_dna,3.33642e-06,2.84466e-05,9210.19,2.44514,0.168458,0.100743,0.0288573,0.0190806,0.0119287,0.00664705,...,0,0,0,0,0,0,0,0,0,0
kbest_dna,4.18096,2.7331,24.326,15.8712,11.6297,12.9221,7.52272,2.83545,2.01466,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
trees_dna,0.0028157,0.00230724,0.00286827,0.00181727,0.000378107,0.00101271,0.000174067,0,0.000196626,0,...,0,0,0,0,0,0,0,0,0,0
var_dnabase,1.19701e-06,2.21533e-05,12129.5,3.87146,0.207323,0.157578,0.0282114,0.0123874,0.0195559,0.00888028,...,0,0,0,0,0,0,0,0,0,0
kbest_dnabase,0.840089,1.32965,10.3328,8.02974,5.4358,5.5394,6.40258,1.30042,0.823026,0.201158,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
trees_dnabase,0.00137727,0.000657109,0.00286335,0.00125877,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
var_dnabackbone,1.29903e-06,2.77124e-05,8473.72,2.0384,0.164327,0.11186,0.026,0.0173926,0.00795904,0.003984,...,0,0,0,0,0,0,0,0,0,0
kbest_dnabackbone,7.19807,5.09115,19.8228,12.5397,10.8301,6.89983,5.21337,6.64812,3.62323,0.200514,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
trees_dnabackbone,0.00206842,0.00315759,0.00416463,0.000965906,0.0004,0.000922161,0.000118154,0,0,0.000126535,...,0,0,0,0,0,0,0,0,0,0
var_rna,2.55465e-06,3.18085e-05,622.236,0.363502,0.0463577,0.0249418,0.0172833,0.00631886,0.00237905,0.000794281,...,0,0,0,0,0,0,0,0,0,0


In [54]:
## Train and test on all data
def train_test_all(pred_dict, auc_dict, auprc_dict, ligand_bind_features, ligand_name, features=[]):
    class_idx = 1
    
    # Default: Exclude no features
    if len(features) == 0:
        features = np.ones([ligand_bind_features.shape[1],]).astype(bool)
    
    classifiers = {}
    classifiers["Logistic"] = LogisticRegression()
    classifiers["RF"] = ensemble.RandomForestRegressor(n_estimators=1000)  
    classifiers["KNN"] = neighbors.KNeighborsRegressor(n_neighbors=100)
    classifiers["Lasso"] = linear_model.Lasso()
    classifiers["Ridge"] = linear_model.RidgeClassifier()
    classifiers["SVM"] = svm.SVC(kernel='rbf', probability=True)
    
    predict_th = 0.5
    models_req_scaling = ["SVM", "KNN"]
    
    for classifier in classifiers.keys():
        #print "starting "+ligand_name+": "+classifier
        model = classifiers[classifier]
        auc_sum = 0
        
        #Create X and y by taking all positives and equal # negatives
        positives = ligand_bind_features.iloc[:,features]
        negatives = non_binding_positions.iloc[:,features]
        test_i_neg = np.random.permutation(len(negatives))[0:len(positives)]
        X_subset = pd.concat([positives, negatives.iloc[test_i_neg]])

        if (classifier in models_req_scaling):
            X_subset = pd.DataFrame(scale(X_subset)) #Is z-scoring the data needed?

        y_subset = [1] * len(positives)
        y_subset.extend([0] * len(test_i_neg))
        y_subset = np.array(y_subset)

        #Fit to training data
        model = classifiers[classifier]
        model.fit(X_subset, y_subset)
        probs_list = []
        
        #Get predictions
        if (classifier == "Logistic" or classifier == "SVM"):
            probs = model.predict_proba(X_subset)
            predicted = log_predict_threshold(probs, predict_th, class_idx)
            for l in probs:
                probs_list.append(l[1])
        elif (classifier == "Ridge"):
            probs = model.decision_function(X_subset)
            predicted = model.predict(X_subset)
            probs_list = probs
        else:
            probs = model.predict(X_subset)
            predicted = predict_threshold(probs, predict_th)
            probs_list = probs

        #Save values
        pred_dict["pred"].extend(predicted)
        pred_dict["obs"].extend(y_subset)

        pred_dict["prob"].extend(probs_list)

        fold_list = [1] * len(predicted)
        pred_dict["fold"].extend(fold_list)

        model_list = [classifier] * len(predicted)
        pred_dict["model"].extend(model_list)

        if (classifier == "Logistic" or classifier == "SVM"):
            #print "auc= "+str(metrics.roc_auc_score(y_test, probs[:, 1]))
            auc_dict[classifier].append(metrics.roc_auc_score(y_subset, probs[:, 1]))
            precision, recall, _ = precision_recall_curve(y_subset, probs[:, 1])

        else:
            #print "auc= "+str(metrics.roc_auc_score(y_test, probs))
            auc_dict[classifier].append(metrics.roc_auc_score(y_subset, probs))
            precision, recall, _ = precision_recall_curve(y_subset, probs)
        auprc_dict[classifier].append(auc(recall, precision))

        avg_auc = np.sum(auc_dict[classifier])
        print "avg auc = "+str(avg_auc)
        
        avg_auprc = np.sum(auprc_dict[classifier])
        print "avg auprc = "+str(avg_auprc)
            
        print "Finished "+ligand+" "+classifier

In [56]:
## Run testing and training on all data - separately for all and non-conserved
cons = ["con","noncon"]
old_ligands = ["all_ligands","dnabase","rnabase","peptide","ion","metabolite"]
for c in cons:
    for ligand in old_ligands:
        pred_dict = defaultdict(list)
        auc_dict = defaultdict(list)
        auprc_dict = defaultdict(list)
            
        if ligand == "all_ligands":
            df = pd.concat([ligands_features_df["dnabase"], ligands_features_df["rnabase"], ligands_features_df["ion"],
                            ligands_features_df["peptide"], ligands_features_df["metabolite"]])
        else:
            df = ligands_features_df[ligand]
            
        if c == "noncon":
            df = df[df["pfam_prob_max"] <= 0.5]
            
        train_test_all(pred_dict, auc_dict, auprc_dict, df, ligand)

        pred_df = pd.DataFrame.from_dict(pred_dict)
        auc_df = pd.DataFrame.from_dict(auc_dict)
        auprc_df = pd.DataFrame.from_dict(auprc_dict)

        #Save to file
        pred_df.to_csv(curr_dir[0]+"/ligand_df/"+ligand+"_0.1_test_"+c+".csv", sep='\t')
        auc_df.to_csv(curr_dir[0]+"/ligand_df/"+ligand+"_0.1_test_"+c+"_auc.csv", sep='\t')
        auprc_df.to_csv(curr_dir[0]+"/ligand_df/"+ligand+"_0.1_test_"+c+"_auprc.csv", sep='\t')

avg auc = 0.673805034631
avg auprc = 0.688456845318
Finished all_ligands KNN
avg auc = 0.775849309203
avg auprc = 0.78512608896
Finished all_ligands SVM
avg auc = 0.701158086388
avg auprc = 0.724865243721
Finished all_ligands Ridge
avg auc = 1.0
avg auprc = 1.0
Finished all_ligands RF
avg auc = 0.597126615604
avg auprc = 0.619438434033
Finished all_ligands Logistic
avg auc = 0.583858903483
avg auprc = 0.604171691209
Finished all_ligands Lasso
avg auc = 0.719818927916
avg auprc = 0.745263122271
Finished dnabase KNN
avg auc = 0.909636134068
avg auprc = 0.917284037215
Finished dnabase SVM
avg auc = 0.944186179982
avg auprc = 0.949730207358
Finished dnabase Ridge
avg auc = 1.0
avg auprc = 1.0
Finished dnabase RF
avg auc = 0.704401974288
avg auprc = 0.736250238342
Finished dnabase Logistic
avg auc = 0.736520029844
avg auprc = 0.765882828017
Finished dnabase Lasso
avg auc = 0.677066802222
avg auprc = 0.663335877658
Finished rnabase KNN
avg auc = 0.900267804007
avg auprc = 0.899188801499
Fini

### Classifier for a particular ligand-binding
#### Test different algorithms with features removed

In [39]:
def test_model(pred_dict, auc_dict, auprc_dict, ligand_bind_features, ligand_name, features=[]):
    class_idx = 1
    
    # Default: Exclude no features
    if len(features) == 0:
        features = np.ones([ligand_bind_features.shape[1],]).astype(bool)
    
    classifiers = {}
    classifiers["Logistic"] = LogisticRegression()
    classifiers["RF"] = ensemble.RandomForestRegressor(n_estimators=1000)  
    classifiers["KNN"] = neighbors.KNeighborsRegressor(n_neighbors=100)
    classifiers["Lasso"] = linear_model.Lasso()
    classifiers["Ridge"] = linear_model.RidgeClassifier()
    classifiers["SVM"] = svm.SVC(kernel='rbf', probability=True)
    
    predict_th = 0.5
    models_req_scaling = ["SVM", "KNN"]
    
    for classifier in classifiers.keys():
        #print "starting "+ligand_name+": "+classifier
        model = classifiers[classifier]
        auc_sum = 0
        #Create X and y
        X = pd.concat([ligand_bind_features.iloc[:,features], non_binding_positions.iloc[:,features]])
        #X = pd.concat([ligand_bind_features, non_binding_positions])
        
        if (classifier in models_req_scaling):
            X = pd.DataFrame(scale(X)) #Is z-scoring the data needed?

        y = [1] * ligand_bind_features.shape[0]
        y.extend([0] * non_binding_positions.shape[0])
        y = np.array(y)

        binding_skf = StratifiedKFold(n_splits=10)
        pred_idx = 1

        for train_index, test_index in binding_skf.split(X, y):
            X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
            y_train, y_test = y[train_index], y[test_index]

            #Down-sample negative examples to have balanced classes
            negative_idx = np.where(y_train == 0)[0].tolist()
            positive_idx = np.where(y_train == 1)[0].tolist()
            sampled_negative_idx = sample(iter(negative_idx), len(positive_idx))
            X_train_sampled = pd.concat([X_train.iloc[positive_idx], X_train.iloc[sampled_negative_idx]])
            y_train_sampled = np.append(y_train[positive_idx], [0] * len(positive_idx))
            
            #fit to training data
            model = classifiers[classifier]
            model.fit(X_train_sampled, y_train_sampled)
            probs_list = []

            if (classifier == "Logistic" or classifier == "SVM"):
                probs = model.predict_proba(X_test)
                predicted = log_predict_threshold(probs, predict_th, class_idx)
                for l in probs:
                    probs_list.append(l[1])
            elif (classifier == "Ridge"):
                probs = model.decision_function(X_test)
                predicted = model.predict(X_test)
                probs_list = probs
            else:
                probs = model.predict(X_test)
                predicted = predict_threshold(probs, predict_th)
                probs_list = probs

            #predicted = model.predict(X_test)

            pred_dict["pred"].extend(predicted)
            pred_dict["obs"].extend(y_test)

            pred_dict["prob"].extend(probs_list)

            fold_list = [pred_idx] * len(predicted)
            pred_dict["fold"].extend(fold_list)

            model_list = [classifier] * len(predicted)
            pred_dict["model"].extend(model_list)

            if (classifier == "Logistic" or classifier == "SVM"):
                #print "auc= "+str(metrics.roc_auc_score(y_test, probs[:, 1]))
                auc_dict[classifier].append(metrics.roc_auc_score(y_test, probs[:, 1]))
                precision, recall, _ = precision_recall_curve(y_test, probs[:, 1])
                
            else:
                #print "auc= "+str(metrics.roc_auc_score(y_test, probs))
                auc_dict[classifier].append(metrics.roc_auc_score(y_test, probs))
                precision, recall, _ = precision_recall_curve(y_test, probs)
            auprc_dict[classifier].append(auc(recall, precision))

            pred_idx += 1

        avg_auc = np.sum(auc_dict[classifier])/10.0
        print "avg auc = "+str(avg_auc)
        
        avg_auprc = np.sum(auprc_dict[classifier])/10.0
        print "avg auprc = "+str(avg_auprc)
            
        print "Finished "+ligand+" "+classifier

In [39]:
# Models with features removed
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import ExtraTreesClassifier

kbest_list = [150]
trees_list = [0.003]

for k_thresh in kbest_list:
    for t_thresh in trees_list:
        for ligand in ligands:
            #Initialize dictionary
            pred_dict = defaultdict(list)
            auc_dict = defaultdict(list)
            auprc_dict = defaultdict(list)

            # Remove features
            X,y = get_sample(ligands_features_df[ligand],non_binding_positions)
            
            # Remove 0 variance features first
            var_model = VarianceThreshold(threshold=0).fit(X)
            X = X.loc[:,var_model.get_support()]

            # Kbest
            kbest_model = SelectKBest(k=k_thresh).fit(X,y)
            kbest_included = kbest_model.get_support()

            # Trees
            trees_model = ExtraTreesClassifier().fit(X,y)
            trees_included = (trees_model.feature_importances_ > t_thresh)
            
            # Combine exclusions
            features_included = (kbest_included & trees_included)

            # Run classifier
            test_model(pred_dict, auc_dict, auprc_dict, ligands_features_df[ligand], ligand, features_included)

            pred_df = pd.DataFrame.from_dict(pred_dict)
            auc_df = pd.DataFrame.from_dict(auc_dict)

            #Save to file
            #pred_df.to_csv(curr_dir[0]+"/feature_select_df/"+ligand+"_0.1_trees_"+str(thresh)+".csv", sep='\t')
            #auc_df.to_csv(curr_dir[0]+"/feature_select_df/"+ligand+"_0.1_auc_trees_"+str(thresh)+".csv", sep='\t')

            print "Finished ligand "+ligand

Finished ligand dna
Finished ligand dnabase
Finished ligand dnabackbone
Finished ligand rna
Finished ligand rnabase
Finished ligand rnabackbone
Finished ligand peptide
Finished ligand ion
Finished ligand metabolite
